# Daytime Hypotesis for fft


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import stats


def analyze_fft_structure(loaded):
    """Анализирует структуру FFT данных для exec/rest периодов"""
    power_example = loaded['power_0']
    
    print(f"Shape of power data: {power_example.shape}")
    print(f"Number of dimensions: {power_example.ndim}")
    
    if power_example.ndim == 2:
        if power_example.shape[0] == 64 or power_example.shape[0] == 63:
            n_channels = power_example.shape[0]
            n_freq_bins = power_example.shape[1]
            data_structure = "(channels, frequencies)"
        else:
            n_channels = 1
            n_freq_bins = power_example.shape[1]
            data_structure = "(time, frequencies) - single channel"
    
    elif power_example.ndim == 3:
        n_channels = power_example.shape[0]
        n_time_points = power_example.shape[1]
        n_freq_bins = power_example.shape[2]
        data_structure = "(channels, time, frequencies)"
    
    else:
        raise ValueError(f"Unexpected data dimension: {power_example.ndim}")
    
    print(f"Data structure: {data_structure}")
    print(f"Number of channels: {n_channels}")
    print(f"Number of frequency bins: {n_freq_bins}")
    
    freqs = np.linspace(0, 100, n_freq_bins)
    
    return freqs, n_channels, n_freq_bins

def process_excel_time(file_path, column_index=19):
    """Обрабатывает временные категории из Excel файла"""
    df = pd.read_excel(file_path, header=1)
    time_data = df.iloc[:, column_index]
    
    time_category = []
    for cell in time_data:
        if pd.isna(cell):
            time_category.append(None)
        else:
            try:
                if isinstance(cell, str):
                    try:
                        dt = datetime.strptime(cell, "%d.%m.%Y %H:%M")
                    except ValueError:
                        dt = datetime.strptime(cell, "%Y-%m-%d %H:%M:%S")
                else:
                    dt = cell if hasattr(cell, 'hour') else pd.to_datetime(cell)
                
                time_category.append("day" if dt.hour < 18 else "evening")
            except Exception as e:
                print(f"Ошибка обработки ячейки {cell}: {e}")
                time_category.append(None)
    
    return time_category

def extract_exec_rest_periods(power_data, data_dims):
    """Корректно извлекает exec и rest периоды с учетом структуры данных"""
    
    if data_dims == '3d':
        # 3D данные: (каналы, время, частоты)
        n_time_points = power_data.shape[1]
        
        if n_time_points >= 6:
            rest1_end = n_time_points // 3
            exec_end = 2 * n_time_points // 3
            
            periods = {
                'rest1': slice(0, rest1_end),
                'exec': slice(rest1_end, exec_end),
                'rest2': slice(exec_end, n_time_points),
                'full': slice(0, n_time_points)
            }
        else:
            periods = {'full': slice(0, n_time_points)}
    
    elif data_dims == '2d_temporal':
        # 2D временные данные: (время, частоты)
        n_time_points = power_data.shape[0]
        
        if n_time_points >= 6:
            rest1_end = n_time_points // 3
            exec_end = 2 * n_time_points // 3
            
            periods = {
                'rest1': slice(0, rest1_end),
                'exec': slice(rest1_end, exec_end),
                'rest2': slice(exec_end, n_time_points),
                'full': slice(0, n_time_points)
            }
        else:
            periods = {'full': slice(0, n_time_points)}
    
    elif data_dims == '2d_spectral':
        # 2D спектральные данные: (каналы, частоты) - нет временной динамики
        periods = {'full': slice(0, power_data.shape[1])}
    
    else:
        raise ValueError(f"Unsupported data dimensions: {data_dims}")
    
    print(f"  Extracted periods: {list(periods.keys())}")
    return periods

def process_power_data(power_data, periods, channel, band_mask, data_dims):
    """Обрабатывает данные мощности для заданного периода и канала"""
    
    band_powers = {}
    
    for period_name, period_slice in periods.items():
        try:
            if data_dims == '3d':
                # (channels, time, frequencies)
                time_data = power_data[channel, period_slice, :]
                band_power = np.mean(time_data[:, band_mask])
                
            elif data_dims == '2d_temporal':
                # (time, frequencies) 
                time_data = power_data[period_slice, :]
                band_power = np.mean(time_data[:, band_mask])
                
            elif data_dims == '2d_spectral':
                # (channels, frequencies) - нет временной динамики
                channel_data = power_data[channel, :]
                band_power = np.mean(channel_data[band_mask])
                
            else:
                band_power = np.nan
                
            band_powers[period_name] = band_power
            
        except Exception as e:
            print(f"    Error processing {period_name}: {e}")
            band_powers[period_name] = np.nan
    
    return band_powers

def perform_frequency_specific_analysis(raw_df, significant_results, freqs, results_arr):
    """Выполняет анализ для отдельных частот и окон 2-3 Hz для значимых результатов"""
    print(f"\nPerforming frequency-specific analysis for {len(significant_results)} significant results...")
    
    frequency_results = []
    
    for idx, sig_result in enumerate(significant_results):
        period_type = sig_result['period_type']
        channel = sig_result['channel']
        freq_band = sig_result['frequency_band']
        
        print(f"  Processing significant result {idx+1}/{len(significant_results)}: {period_type}, channel {channel}, {freq_band}")
        
        # Фильтруем данные для этого конкретного случая
        subset_data = raw_df[
            (raw_df['period_type'] == period_type) &
            (raw_df['channel'] == channel) &
            (raw_df['frequency_band'] == freq_band)
        ]
        
        if len(subset_data) == 0:
            continue
            
        # Получаем subject_id и time_category для анализа
        subject_ids = subset_data['subject_id'].unique()
        
        for subject_id in subject_ids[:5]:  # Ограничим для скорости
            subject_data = subset_data[subset_data['subject_id'] == subject_id]
            if len(subject_data) == 0:
                continue
                
            time_category = subject_data['time_category'].iloc[0]
            
            # Находим соответствующие исходные FFT данные
            for record in results_arr:
                power_data, s_id, t_id, gender, handiness, age, label, img, task_type = record
                if s_id != subject_id:
                    continue
                
                # Определяем структуру данных
                if power_data.ndim == 3:
                    data_dims = '3d'
                elif power_data.ndim == 2:
                    if power_data.shape[0] == 64 or power_data.shape[0] == 63:
                        data_dims = '2d_spectral'
                    else:
                        data_dims = '2d_temporal'
                else:
                    continue
                
                # Анализ для отдельных частот в диапазоне 2-3 Hz
                freq_mask_2_3 = (freqs >= 2) & (freqs < 3)
                target_freqs = freqs[freq_mask_2_3]
                freq_indices = np.where(freq_mask_2_3)[0]
                
                for freq_idx, freq in zip(freq_indices[:3], target_freqs[:3]):  # Первые 3 частоты
                    try:
                        if data_dims == '3d':
                            # Для 3D данных используем весь временной период
                            real_power = np.mean(power_data[channel, :, freq_idx])
                            
                        elif data_dims == '2d_temporal':
                            real_power = np.mean(power_data[:, freq_idx])
                            
                        elif data_dims == '2d_spectral':
                            real_power = power_data[channel, freq_idx]
                        
                        frequency_results.append({
                            'original_comparison_idx': idx,
                            'period_type': period_type,
                            'channel': channel,
                            'original_frequency_band': freq_band,
                            'specific_frequency': freq,
                            'subject_id': subject_id,
                            'time_category': time_category,
                            'real_power': real_power,
                            'data_dimensionality': data_dims
                        })
                        
                    except Exception as e:
                        continue
        
        if (idx + 1) % 10 == 0:
            print(f"  Processed {idx + 1}/{len(significant_results)} significant results")
    
    return pd.DataFrame(frequency_results)


# Load data

In [3]:

print("Loading data...")
loaded = np.load('C:/Users/HP/Documents/GitHub/EEG-Image-Reconstruction/Generated/Spectrums/exec_and_rest_fft.npz')

# Создаем список для хранения всех данных
results_arr = []

print("Reading records from npz file...")
i = 0
while f'power_{i}' in loaded:
    power = loaded[f'power_{i}']
    s_id = int(loaded[f'subject_id_{i}'])
    t_id = int(loaded[f'trial_id_{i}'])
    gender = str(loaded[f'gender_{i}'])
    handiness = str(loaded[f'handiness_{i}'])
    age = int(loaded[f'age_{i}'])
    label = int(loaded[f'label_{i}'])
    img = loaded[f'img_{i}']
    task_type = str(loaded[f'task_type_{i}'])
    
    results_arr.append([power, s_id, t_id, gender, handiness, age, label, img, task_type])
    i += 1

print(f"Loaded {len(results_arr)} records")

# Анализируем структуру power данных
freqs, n_channels, n_freq_bins = analyze_fft_structure(loaded)
print(f"Frequency range: {freqs[0]:.1f} - {freqs[-1]:.1f} Hz")

# Загружаем временные категории
print("Loading time categories...")
time_categories = process_excel_time('C:/Users/HP/Documents/GitHub/EEG-Image-Reconstruction/Supplementary/Experiment_Metadata.xlsx', column_index=19)

# Определяем частотные диапазоны EEG
freq_bands = {
    'delta': (0, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 80)
}

# Создаем список для результатов
results = []


Loading data...
Reading records from npz file...
Loaded 651 records
Shape of power data: (63, 381)
Number of dimensions: 2
Data structure: (channels, frequencies)
Number of channels: 63
Number of frequency bins: 381
Frequency range: 0.0 - 100.0 Hz
Loading time categories...


# Processing of all records

In [4]:
print(f"\nProcessing {len(results_arr)} power records...")

for i, record in enumerate(results_arr):
    try:
        power_data, s_id, t_id, gender, handiness, age, label, img, task_type = record
        
        if s_id - 1 >= len(time_categories) or time_categories[s_id - 1] is None:
            print(f"Record {i}: No time category for subject {s_id}")
            continue
        
        time_cat = time_categories[s_id - 1]
        
        print(f"Record {i}: subject {s_id}, shape {power_data.shape}, ndim {power_data.ndim}")
        
        # Определяем структуру данных
        if power_data.ndim == 3:
            data_dims = '3d'
            n_channels_current = power_data.shape[0]
        elif power_data.ndim == 2:
            if power_data.shape[0] == 64 or power_data.shape[0] == 63:
                data_dims = '2d_spectral'
                n_channels_current = power_data.shape[0]
            else:
                data_dims = '2d_temporal'
                n_channels_current = 1
        else:
            print(f"  Unsupported dimensionality: {power_data.ndim}")
            continue
        
        # Извлекаем периоды с учетом структуры данных
        periods = extract_exec_rest_periods(power_data, data_dims)
        
        # Обрабатываем каждый канал и частотный диапазон
        for channel in range(n_channels_current):
            for band_name, (f_low, f_high) in freq_bands.items():
                band_mask = (freqs >= f_low) & (freqs < f_high)
                
                # Получаем мощности для всех периодов
                band_powers = process_power_data(
                    power_data, periods, channel, band_mask, data_dims
                )
                
                # Сохраняем результаты для каждого периода
                for period_name, band_power in band_powers.items():
                    if not np.isnan(band_power):
                        results.append({
                            'subject_id': s_id,
                            'trial_id': t_id,
                            'time_category': time_cat,
                            'period_type': period_name,
                            'channel': channel,
                            'frequency_band': band_name,
                            'power': band_power,
                            'gender': gender,
                            'handiness': handiness,
                            'age': age,
                            'label': label,
                            'task_type': task_type,
                            'data_dimensionality': data_dims
                        })
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(results_arr)} records")
            
    except Exception as e:
        print(f"Error processing record {i}: {e}")
        continue

print("Power data collection completed!")
print(f"Collected {len(results)} data points")



Processing 651 power records...
Record 0: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 1: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 2: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 3: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 4: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 5: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 6: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 7: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 8: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 9: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Processed 10/651 records
Record 10: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 11: subject 4, shape (63, 381), ndim 2
  Extracted periods: ['full']
Record 12: subject 4, shape (63, 381), n

# Statistical analysis

In [6]:
# Проверяем, есть ли данные для анализа
if len(results) == 0:
    print("No data collected! Check your data structure.")
else:
    # Создаем DataFrame с сырыми данными
    raw_df = pd.DataFrame(results)
    
    # Проверяем распределение данных
    print("\nData distribution:")
    print(f"Total records: {len(raw_df)}")
    print(f"Day records: {len(raw_df[raw_df['time_category'] == 'day'])}")
    print(f"Evening records: {len(raw_df[raw_df['time_category'] == 'evening'])}")
    print(f"Period types distribution:")
    period_counts = raw_df['period_type'].value_counts()
    for period, count in period_counts.items():
        day_count = len(raw_df[(raw_df['period_type'] == period) & (raw_df['time_category'] == 'day')])
        evening_count = len(raw_df[(raw_df['period_type'] == period) & (raw_df['time_category'] == 'evening')])
        print(f"  {period}: {count} total ({day_count} day, {evening_count} evening)")
    
    # Статистический анализ: сравнение day vs evening для КАЖДОГО периода
    statistical_results = []
    
    print(f"\nPerforming statistical analysis for ALL period types...")
    
    # АНАЛИЗ ДЛЯ КАЖДОГО ПЕРИОДА ОТДЕЛЬНО: day vs evening
    available_periods = raw_df['period_type'].unique()
    print(f"Available period types for analysis: {available_periods}")
    
    for period in available_periods:
        period_data = raw_df[raw_df['period_type'] == period]
        grouping_columns = ['channel', 'frequency_band']
        
        print(f"\nAnalyzing {period} period (day vs evening):")
        print(f"  Total records: {len(period_data)}")
        print(f"  Day records: {len(period_data[period_data['time_category'] == 'day'])}")
        print(f"  Evening records: {len(period_data[period_data['time_category'] == 'evening'])}")
        
        comparisons_count = 0
        for (channel, band), group in period_data.groupby(grouping_columns):
            day_data = group[group['time_category'] == 'day']['power']
            evening_data = group[group['time_category'] == 'evening']['power']
            
            if len(day_data) >= 3 and len(evening_data) >= 3:
                try:
                    # Используем t-test или Mann-Whitney
                    try:
                        stat, p_value = stats.ttest_ind(day_data, evening_data)
                        test_name = "t-test"
                    except:
                        stat, p_value = stats.mannwhitneyu(day_data, evening_data)
                        test_name = "mannwhitneyu"
                    
                    significant = p_value < 0.05
                    
                    statistical_results.append({
                        'comparison': f'day_vs_evening',
                        'period_type': period,
                        'channel': channel,
                        'frequency_band': band,
                        'test_name': test_name,
                        'statistic': stat,
                        'p_value': p_value,  # Сохраняем чистое p-value
                        'significant': significant,
                        'n_day': len(day_data),
                        'n_evening': len(evening_data),
                        'mean_power_day': np.mean(day_data),
                        'mean_power_evening': np.mean(evening_data),
                        'power_difference': np.mean(day_data) - np.mean(evening_data)
                    })
                    comparisons_count += 1
                    
                except Exception as e:
                    continue
        
        print(f"  Completed {comparisons_count} comparisons for {period}")
    
    # АНАЛИЗ ДЛЯ ОБЪЕДИНЕННЫХ REST ПЕРИОДОВ
    if 'rest1' in available_periods and 'rest2' in available_periods:
        print(f"\nAnalyzing combined rest periods (day vs evening):")
        
        rest_combined_df = raw_df.copy()
        rest_combined_df['period_type_combined'] = rest_combined_df['period_type'].apply(
            lambda x: 'rest_combined' if x in ['rest1', 'rest2'] else x
        )
        
        rest_combined_data = rest_combined_df[rest_combined_df['period_type_combined'] == 'rest_combined']
        grouping_columns = ['channel', 'frequency_band']
        
        comparisons_count = 0
        for (channel, band), group in rest_combined_data.groupby(grouping_columns):
            day_data = group[group['time_category'] == 'day']['power']
            evening_data = group[group['time_category'] == 'evening']['power']
            
            if len(day_data) >= 3 and len(evening_data) >= 3:
                try:
                    # Используем t-test или Mann-Whitney
                    try:
                        stat, p_value = stats.ttest_ind(day_data, evening_data)
                        test_name = "t-test"
                    except:
                        stat, p_value = stats.mannwhitneyu(day_data, evening_data)
                        test_name = "mannwhitneyu"
                    
                    significant = p_value < 0.05
                    
                    statistical_results.append({
                        'comparison': 'day_vs_evening',
                        'period_type': 'rest_combined',
                        'channel': channel,
                        'frequency_band': band,
                        'test_name': test_name,
                        'statistic': stat,
                        'p_value': p_value,
                        'significant': significant,
                        'n_day': len(day_data),
                        'n_evening': len(evening_data),
                        'mean_power_day': np.mean(day_data),
                        'mean_power_evening': np.mean(evening_data),
                        'power_difference': np.mean(day_data) - np.mean(evening_data)
                    })
                    comparisons_count += 1
                    
                except Exception as e:
                    continue
        
        print(f"  Completed {comparisons_count} comparisons for rest_combined")
    
    # Создаем финальный DataFrame
    if statistical_results:
        final_df = pd.DataFrame(statistical_results)
        
        # НЕ ИСПРАВЛЯЕМ данные анализа - оставляем чистые p-value
        n_comparisons = len(final_df)
        if n_comparisons > 0:
            final_df['p_value_corrected'] = final_df['p_value'] * n_comparisons
            final_df['p_value_corrected'] = np.minimum(final_df['p_value_corrected'], 1.0)
            final_df['significant_corrected'] = final_df['p_value_corrected'] < 0.05
            
            # Сохраняем результаты с чистыми p-value
            output_path = 'C:/Users/HP/Documents/GitHub/EEG-Image-Reconstruction/Supplementary/daytime_fft.csv'
            final_df.to_csv(output_path, index=False)
            
            print(f"\nAnalysis completed! Results saved to {output_path}")
            print(f"Total comparisons: {len(final_df)}")
            print(f"Significant results (uncorrected p < 0.05): {final_df['significant'].sum()}")
            print(f"Significant results (corrected p < 0.05): {final_df['significant_corrected'].sum()}")
            
            # Детальная сводка по периодам
            print("\nResults by period type:")
            all_periods_in_results = final_df['period_type'].unique()
            for period in all_periods_in_results:
                period_data = final_df[final_df['period_type'] == period]
                if len(period_data) > 0:
                    sig_uncorrected = period_data['significant'].sum()
                    sig_corrected = period_data['significant_corrected'].sum()
                    print(f"  {period}: {len(period_data)} comparisons, {sig_uncorrected} uncorrected significant, {sig_corrected} corrected significant")
            
            # # Анализ для значимых результатов на отдельных частотах 2-3 Hz
            # significant_results_list = final_df[final_df['significant'] == True].to_dict('records')
            # if len(significant_results_list) > 0:
            #     print(f"\nPerforming detailed frequency analysis for {len(significant_results_list)} significant results...")
            #     frequency_specific_df = perform_frequency_specific_analysis(
            #         raw_df, significant_results_list, freqs, results_arr
            #     )
                
            #     if len(frequency_specific_df) > 0:
            #         freq_output_path = 'C:/Users/HP/Documents/GitHub/EEG-Image-Reconstruction/Supplementary/frequency_specific_analysis_2_3Hz.csv'
            #         frequency_specific_df.to_csv(freq_output_path, index=False)
            #         print(f"Frequency-specific analysis saved to {freq_output_path}")
            #         print(f"Analyzed {len(frequency_specific_df)} frequency-specific data points")
            
            # print("\nFirst 10 results (with all period types):")
            # print(final_df.head(10).to_string())
            
            # # Показываем распределение по всем периодам
            # print(f"\nFinal results include ALL period types: {list(final_df['period_type'].unique())}")
        else:
            print("No statistical results were generated.")
    else:
        print("No statistical results were generated.")

print("\nScript execution completed successfully!")


Data distribution:
Total records: 205065
Day records: 112455
Evening records: 92610
Period types distribution:
  full: 205065 total (112455 day, 92610 evening)

Performing statistical analysis for ALL period types...
Available period types for analysis: ['full']

Analyzing full period (day vs evening):
  Total records: 205065
  Day records: 112455
  Evening records: 92610
  Completed 315 comparisons for full

Analysis completed! Results saved to C:/Users/HP/Documents/GitHub/EEG-Image-Reconstruction/Supplementary/daytime_fft.csv
Total comparisons: 315
Significant results (uncorrected p < 0.05): 246
Significant results (corrected p < 0.05): 158

Results by period type:
  full: 315 comparisons, 246 uncorrected significant, 158 corrected significant

Script execution completed successfully!
